<h1> Masculinity survey </h1>

Import required python modules

In [7]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

Import the masculinity survey data and convert the word answers into numerical values using the mapping_dictionary.
Then drop any Nan records from the survey results.

In [8]:
survey = pd.read_csv("masculinity.csv")
mapping_dictionary = {"Often" : 4, "Sometimes" : 3, "Rarely" : 2, "Never, but open to it" : 1, "Never, and not open to it" : 0.}
column_list = []
for num in range(1,12):
    if num < 10: 
        survey["q0007_000" + str(num)] = survey["q0007_000" + str(num)].map(mapping_dictionary)
        column_list += ["q0007_000" + str(num)]
    else: 
        survey["q0007_00" + str(num)] = survey["q0007_00" + str(num)].map(mapping_dictionary)
        column_list += ["q0007_00" + str(num)]
rows_to_cluster = survey.dropna(subset = column_list)

Create a Kmeans model and fit it to the rows under assesment. n_clusters = 2 as the data will be seperated into 2 groups. One exhibiting more traits associated with toxic masculinity and one not.

In [17]:
model = KMeans(n_clusters = 2)
model.fit(rows_to_cluster[column_list])
print(model.cluster_centers_)

[[1.91052632 1.85263158 0.95789474 1.66578947 0.53947368 2.88421053
  0.08421053 2.80789474 2.17894737 0.60789474 1.66315789]
 [2.85381026 2.83359253 2.83981337 2.44012442 0.71695179 2.74339036
  0.52410575 2.97045101 2.80248834 1.53654743 2.39502333]]


Create 2 lists to seperate the datapoints labeled as 0 (more masculine) and 1 (less masculine)

In [87]:
cluster_zero_indices, cluster_one_indices = [[],[]]
for row in range(len(model.labels_)):
    if model.labels_[row] == 0:
        cluster_zero_indices += [row]
    else: cluster_one_indices += [row]
cluster_zero_df = rows_to_cluster.iloc[cluster_zero_indices]
cluster_one_df = rows_to_cluster.iloc[cluster_one_indices]

Define a function that returns the relative percentage of different attributes making up each masculinity group

In [115]:
def percentage_breakdown_by_attribute(zero_cluster_df, one_cluster_df, attribute):
    cluster_zero_dict = {}
    for row in range(len(list(zero_cluster_df[attribute].value_counts().index))):
        cluster_zero_dict[list(zero_cluster_df[attribute].value_counts().index)[row]] = [str(round(zero_cluster_df[attribute].value_counts().values[row]*100/len(zero_cluster_df),2))+"%"]
    cluster_zero_df_percentages = pd.DataFrame.from_dict(cluster_zero_dict)
    cluster_zero_df_percentages["Group"] = ["More toxic masculinity"]
    cluster_one_dict = {}
    for i in range(len(list(one_cluster_df[attribute].value_counts().index))):
        cluster_one_dict[list(one_cluster_df[attribute].value_counts().index)[i]] = [str(round(one_cluster_df[attribute].value_counts().values[i]*100/len(one_cluster_df),2))+"%"]
    cluster_one_df_percentages = pd.DataFrame.from_dict(cluster_one_dict)
    cluster_one_df_percentages["Group"] = ["Less toxic masculinity"]
    if 'cluster_df_percentages' in locals():
        del cluster_df_percentages
    cluster_df_percentages = pd.concat([cluster_zero_df_percentages,cluster_one_df_percentages]).reset_index()
    cluster_df_percentages.drop(columns = ["index"], inplace = True)
    cluster_df_percentages.set_index("Group", inplace = True)
    pd.DataFrame.transpose(cluster_df_percentages)
    return cluster_df_percentages

Apply the function to different attributes 
<h3> Education level </h3>

In [116]:
percentage_breakdown_by_attribute(cluster_zero_df, cluster_one_df, "educ4")

,College or more,Some college,Post graduate degree,High school or less
Group,,,,
More toxic masculinity,32.11%,29.47%,25.53%,12.63%
Less toxic masculinity,31.26%,23.64%,37.01%,8.09%


<h3> Age </h3>

In [117]:
percentage_breakdown_by_attribute(cluster_zero_df, cluster_one_df, "age3")

,65 and up,35 - 64,18 - 34
Group,,,
More toxic masculinity,53.68%,40.53%,5.53%
Less toxic masculinity,42.46%,50.23%,7.31%


<h3> Race </h3>

In [118]:
percentage_breakdown_by_attribute(cluster_zero_df, cluster_one_df, "race2")

,White,Non-white
Group,,
More toxic masculinity,85.26%,14.74%
Less toxic masculinity,84.91%,15.09%


<h3> Whether they have kids </h3>

In [119]:
percentage_breakdown_by_attribute(cluster_zero_df, cluster_one_df, "kids")

,Has children,No children
Group,,
More toxic masculinity,70.26%,29.21%
Less toxic masculinity,66.72%,32.97%


<h3> Orientation </h3>

In [120]:
percentage_breakdown_by_attribute(cluster_zero_df, cluster_one_df, "orientation")

,Straight,Gay/Bisexual,Other,No answer
Group,,,,
More toxic masculinity,94.47%,3.95%,0.79%,0.53%
Less toxic masculinity,82.27%,15.09%,2.49%,0.16%
